In [ ]:
import raserManipulation as RM
import myServices as ms
import os
import rasterio as rst

In [ ]:
## Testing reprojection
cwd ='/Users/abdielfer/RNCanFloodProject/CrossCanFloodSuscMap/DataSourcesByAOI/TransitDir'
rasterTool = RM.generalRasterTools(cwd)

sourcName = 'dtm_1m_utm13_e_4_159.tif'
sourcePath = os.path.join(cwd,sourcName)
url = 'https://ftp.maps.canada.ca/pub/elevation/dem_mne/highresolution_hauteresolution/dtm_mnt/1m/NS/Urbania/utm20/dtm_1m_utm20_w_2_101.tif'
inName = 'SK_Regina_2019_2ForTest_5m.tif'
inputPath = os.path.join(cwd,inName)

outName = 'SK_Regina_2019_2ForTest_5m_CRS.tif'
outPath = os.path.join(cwd,outName)

crs,trns = rasterTool.get_CRSAndTranslation_GTIFF(url) 
# _,trns = rasterTool.get_CRSAndTranslation_GTIFF(sourcePath)
# res = rasterTool.get_rasterResolution(inputPath)
# print ('original CRS & transformatin', crs , '\n', trns)
# rasterTool.set_CRS_GTIF(inputPath, outPath, crs)

In [ ]:
#### Go through all dir in a root @searchDir:
#####.    check some condition to: 
######.      apply action.

### NO BORRAR. ## DO NOT DELETE

searchDir ='/Users/abdielfer/RNCanFloodProject/CrossCanFloodSuscMap/DataSourcesByAOI'
rasterTool = RM.generalRasterTools(cwd)
for (root, dirs, file) in os.walk(searchDir):
        for i in file:
            if '.csv' in i:
                csvPath = os.path.join(root,i)
                print('CSV found at: ', csvPath)
                dtmFtpList = ms.createListFromCSVColumn(csvPath,'Ftp_dtm')
                newWDir = root
                dtm_5mTif = ms.listFreeFilesInDirByExt(root,'5m.tif')
                dtm_5mTif_path = os.path.join(root,dtm_5mTif[0])
                print('source DTM 5m:', dtm_5mTif_path)
                RM.setWBTWorkingDir(newWDir)
                crs,_=rasterTool.get_CRSAndTranslation_GTIFF(dtmFtpList[0])
                name,_= ms.splitFilenameAndExtention(dtm_5mTif[0])
                dtm_5mTif_CRS_OK = ms.makePath(root,(name +'_CRS_OK.tif'))
                print('destination DTM_5m_CRS_OK:', dtm_5mTif_CRS_OK)
                rasterTool.set_CRS_GTIF(dtm_5mTif_path, dtm_5mTif_CRS_OK, crs)
                print('-------------------------------------------------------')
                

In [ ]:
trans = newTrns['transform']
print(trans[0])
newTrans = rst.Affine(5,trans[1],trans[2],trans[3],-5,trans[5] )


In [ ]:
#### Print UTM zone by Ftp_dtm address in *.csv
cwd = '/Users/abdielfer/RNCanFloodProject/CrossCanFloodSuscMap/DataSourcesByAOI'
file_list = ms.listFreeFilesInDirByExt(cwd, ext ='.csv')
for i in file_list:
    print(i) 
    path = os.path.join(cwd,i)
    with open(path,'r+') as file:
#         reader = csv.reader(file)
        for r in file:
            init = r.find("utm")
            if init > 0:
                end = init+5  
                print(r[init:end])


In [ ]:
### Read ALL *.CSV in cwd, download all DTM_htp by *.CSV, compute Mosaik and Resampling.

cwd ='/Users/abdielfer/RNCanFloodProject/CrossCanFloodSuscMap/DataSourcesByAOI/'

file_list = ms.listFreeFilesInDirByExt(cwd, ext ='.csv')
print(file_list)
rasterTool = RM.generalRasterTools(cwd)
for file in file_list:
    rasterTool.mosaikAndResamplingFromCSV(file,5,'Ftp_dtm',clearTransitDir = True)
    

In [ ]:
# Excecute ONLY RESAMPLING from TransitDir
transitFolderPath ='/Users/abdielfer/RNCanFloodProject/CrossCanFloodSuscMap/DataSourcesByAOI/TransitDir'
k = ms.listFreeFilesInDirByExt(transitFolderPath, ext = '.tif')
name,ext = ms.splitFilenameAndExtention(k[0])
rasterTool = RM.generalRasterTools(transitFolderPath)
resampledRaster = ms.makePath(transitFolderPath,(name + '_5m.tif'))
rasterTool.rasterResampler(k, resampledRaster, 5)

In [ ]:
## Download individuals files
dtmFtpList = ['https://ftp.maps.canada.ca/pub/elevation/dem_mne/highresolution_hauteresolution/dtm_mnt/1m/NRCAN/GTA2015/utm17/dtm_1m_utm17_e_10_82.tif']
transitFolderPath ='/Users/abdielfer/RNCanFloodProject/CrossCanFloodSuscMap/DataSourcesByAOI/TransitDir'
RM.downloadTailsToLocalDir(dtmFtpList,transitFolderPath)

In [ ]:
# Excecute ONLY MOSAIK from TransitDir
transitFolderPath = '/Users/abdielfer/RNCanFloodProject/CrossCanFloodSuscMap/DataSourcesByAOI/TransitDir'
dtmTail = ms.listFreeFilesInDirByExt(transitFolderPath, ext ='.tif')
print(dtmTail[0])
generalRasterTools = RM.generalRasterTools(transitFolderPath)
name,ext = ms.splitFilenameAndExtention(dtmTail[0])
mosaikFileNamePermanent = ms.makePath(transitFolderPath,(name + '_mosaik.tif'))
generalRasterTools.computeMosaic(mosaikFileNamePermanent)

In [ ]:
generalRasterTools.rasterToVectorLine('basin4_StrahOrder_level5.tif', 'basin4_SOL5.shp')

In [ ]:
generalRasterTools.rasterVisibility_index('basin4_5m_fill.tif', 'visibilityBasin4.tif')